In [1]:
import numpy as np 
import scipy.linalg as sla
import matplotlib.pyplot as plt

%matplotlib inline

# Проект по вычислительной математике, 6 семестр

$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) + \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) = -\dfrac{\pi^2}{4}\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}$, $x \in (0, 1)$, $y \in (0, 1)$

$u(0, y) = u(1, y) = u(x, 0) = u(x, 1) = 0$

## Аналитическое решение

Сделаем замену $v = u^2$. Тогда $v_{xx}+v_{yy}=-\dfrac{\pi^2}{2}\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}$

Будем искать решение в виде $v(x, y) = A\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}$. Подстановка в исходное уравнение дает: 

$A\left(-\dfrac{\pi^2}{4}-\dfrac{\pi^2}{4}\right)\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}=-\dfrac{\pi^2}{2}\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}$

Отсюда $A = 1$

Таким образом, $v = \sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}$; $u = \sqrt{\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}}$. Граничные условия выполнены, значит, по теореме Коши это и есть единственно решение уравнения.

Сразу вычислим значения аналитического решения в узлах сетки с шагом $h=0.2$

In [191]:
true_grid = np.zeros((size, size))

for i in range(size):
    for j in range(size):
        true_grid[i][j] = np.sqrt(np.sin(np.pi * i * h / 2) * np.sin(np.pi * j * h / 2))
        
print(true_grid)

[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.30901699 0.42618732 0.5        0.54211865 0.55589297]
 [0.         0.42618732 0.58778525 0.68958557 0.74767439 0.76667154]
 [0.         0.5        0.68958557 0.80901699 0.87716639 0.89945372]
 [0.         0.54211865 0.74767439 0.87716639 0.95105652 0.97522127]
 [0.         0.55589297 0.76667154 0.89945372 0.97522127 1.        ]]


## Метод простой итерации

### Шаг 1. Построение разностной схемы

Аппроксимируем производные по схеме "крест". Узловые точки: $u_{m,n}$, где $n, m \in \{0, 1, 2, ..., N\}$, $N = \frac{1}{h}$, $h$ - шаг сетки, одинаковый по обоим измерениям. 

$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) \approx \dfrac{1}{h}\left(u_{m+0.5,n} \cdot \dfrac{u_{m+1,n} - u_{m,n}}{h} - u_{m-0.5,n} \cdot \dfrac{u_{m,n} - u_{m-1,n}}{h} \right)$

$\dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) \approx \dfrac{1}{h}\left(u_{m,n+0.5} \cdot \dfrac{u_{m,n+1} - u_{m,n}}{h} - u_{m,n-0.5} \cdot \dfrac{u_{m,n} - u_{m,n-1}}{h} \right)$

Точки в половинных узлах аппроксимируем средним арифметическим значений в ближайших узлах. Например, $u_{m+0.5,n} = \dfrac{u_{m,n} + u_{m+1,n}}{2}$. Тогда для производных получим:

$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) \approx \dfrac{1}{2h^2}\left(u_{m+1,n}^2 + u_{m-1,n}^2 - 2u_{m,n}^2 \right)$

$\dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) \approx \dfrac{1}{2h^2}\left(u_{m,n+1}^2 + u_{m,n-1}^2 - 2u_{m,n}^2 \right)$

Разностная задача: 
    
$ \dfrac{1}{2h^2}\left(u_{m+1,n}^2 + u_{m-1,n}^2 - 2u_{m,n}^2 \right) + \dfrac{1}{2h^2}\left(u_{m,n+1}^2 + u_{m,n-1}^2 - 2u_{m,n}^2 \right) = -\dfrac{\pi^2}{4}\sin\dfrac{\pi x_m}{2}\sin\dfrac{\pi y_n}{2}$

Здесь $x_m = hm$, $y_n = hn$.

$u_{m, 0} = u_{0, n} = 0;\quad u_{m, N} = \sqrt{\sin\dfrac{\pi x_m}{2}};\quad u_{N, n} = \sqrt{\sin\dfrac{\pi y_n}{2}}$ 
для любых допустимых значений $m$ и $n$.

### Шаг 2. Постановка нестационарной задачи

Вместо стационарной рассмотрим нестационарную задачу: 

$\dfrac{\partial u}{\partial t} = \dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) + \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) + \dfrac{\pi^2}{4}\sin\dfrac{\pi x}{2}\sin\dfrac{\pi y}{2}$

Ее решение при достаточно большом $t$ сходится к решению нестационарной задачи. 

Решим нестационарную задачу $\textbf{методом простых итераций}$. Для этого аппроксимируем производную по времени: 

$\dfrac{\partial u}{\partial t} = \dfrac{u^{s+1} - u^s}{\tau}$

Здесь $s$ - узел сетки по времени и одновременно порядок итерации, $\tau$ - временной шаг.

Таким образом, нестационарная разностная задача:
    
$\dfrac{u^{s+1}_{m,n} - u^s_{m,n}}{\tau} = \dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{\pi^2}{4}\sin\dfrac{\pi x_m}{2}\sin\dfrac{\pi y_n}{2}$

$u_{m, 0} = u_{0, n} = 0;\quad u_{m, N} = \sqrt{\sin\dfrac{\pi x_m}{2}};\quad u_{N, n} = \sqrt{\sin\dfrac{\pi y_n}{2}}$ 
для любых допустимых значений $m$ и $n$.

Апдейт на одной итерации в узле $(m,n)$:

$u^{s+1}_{m,n} = u^s_{m,n} + \dfrac{\pi^2}{4}\sin\dfrac{\pi x_m}{2}\sin\dfrac{\pi y_n}{2} + \dfrac{\tau}{2h^2}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2 - 4(u^s_{m,n})^2 \right)$

### Шаг 3. Реализация алгоритма

Шаг сетки выберем равным $h = 0.2$. 

Вычислим $\lambda_{min}$ и $\lambda_{max}$ - минимальное и максимальное собственные значения оператора Лапласа. В дальнейшем они будут использованы для выбора $\tau$ и количества итераций метода:

$\lambda_{min} = 2\pi^2$, $\lambda_{max} = \dfrac{8}{h^2} - 2\pi^2$

Метод простых итераций сходится, если выполнено условие: $\tau < \dfrac{2}{\lambda_{max}}$.

Выберем $\tau = \dfrac{2}{\lambda_{min} + \lambda_{max}}$.

In [199]:
h = 0.2
lambda_min = 2 * np.pi ** 2
lambda_max = (8 / h ** 2) - 2 * np.pi
tau = 2 / (lambda_min + lambda_max)

In [200]:
def rhs(m, n):
    ''' Calculates the RHS of the equation in the point (m, n) '''
    return -np.pi ** 2 / 4 * np.sin(np.pi * h * m / 2) * np.sin(np.pi * h * n / 2)

In [204]:
def simple_iter_update(prev_grid, m, n):
    ''' Updates the value in the point (m, n) according to the simple iteration method '''
    combination = (prev_grid[m - 1][n]) ** 2 + \
                  (prev_grid[m + 1][n]) ** 2 + \
                  (prev_grid[m][n - 1]) ** 2 + \
                  (prev_grid[m][n + 1]) ** 2 - \
                  4 * (prev_grid[m][n]) ** 2
    return prev_grid[m][n] - tau * rhs(m, n) + (tau / (2 * h ** 2)) * combination

In [222]:
def solve(update_func, tau=tau, h=h, tol=1e-4, max_iter=1000):
    ''' Solves given equation using an update function update_func '''
    size = int(1 / h + 1)
    prev_grid = np.zeros((size, size))

    # зададим граничные условия
    for i in range(size):
        prev_grid[i][size - 1] = np.sqrt(np.sin(np.pi * h * i / 2))
        prev_grid[size - 1][i] = np.sqrt(np.sin(np.pi * h * i / 2))

    for _ in range(max_iter):
        cur_grid = np.zeros((size, size))
        
        for i in range(size):
            cur_grid[i][size - 1] = np.sqrt(np.sin(np.pi * h * i / 2))
            cur_grid[size - 1][i] = np.sqrt(np.sin(np.pi * h * i / 2))

        for row in range(1, size - 1):
            for col in range(1, size - 1):
                cur_grid[row][col] = update_func(prev_grid, row, col)
        
        if (sla.norm(cur_grid - prev_grid) < tol):
            break
        prev_grid = cur_grid

    return cur_grid

In [209]:
simple_iter_grid = solve(simple_iter_update)
print(simple_iter_grid)

[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.30923671 0.42658638 0.50047858 0.54249953 0.55589297]
 [0.         0.42658638 0.58839733 0.69027185 0.74820575 0.76667154]
 [0.         0.50047858 0.69027185 0.80976402 0.87774192 0.89945372]
 [0.         0.54249953 0.74820575 0.87774192 0.9515094  0.97522127]
 [0.         0.55589297 0.76667154 0.89945372 0.97522127 1.        ]]


## Метод Якоби

### Шаг 1. Постановка разностной задачи

Рассмотрим разностную задачу:

$\dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^{s+1}_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^{s+1}_{m,n})^2 \right) = - \dfrac{\pi^2}{4}\sin\dfrac{\pi x_m}{2}\sin\dfrac{\pi y_n}{2}$

Выразив новое значение в узле, получаем:

$(u^{s+1}_{m,n})^2 = \dfrac{h^2}{2} \dfrac{\pi^2}{4}\sin\dfrac{\pi x_m}{2}\sin\dfrac{\pi y_n}{2}+ \dfrac{1}{4}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2\right)$

### Шаг 2. Реализация алгоритма

In [223]:
def jacobi_update(prev_grid, m, n):
    ''' Updates the value in the point (m, n) according to the Jacobi method '''
    combination = (prev_grid[m - 1][n]) ** 2 + \
                  (prev_grid[m + 1][n]) ** 2 + \
                  (prev_grid[m][n - 1]) ** 2 + \
                  (prev_grid[m][n + 1]) ** 2
    return np.sqrt(combination / 4 - h ** 2 / 2 * rhs(m, n))

In [215]:
jacobi_grid = solve(jacobi_update)
print(jacobi_grid)

[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.30945966 0.42676433 0.50058984 0.54254975 0.55589297]
 [0.         0.42676433 0.58854255 0.69035894 0.74824537 0.76667154]
 [0.         0.50058984 0.69035894 0.80981617 0.87776354 0.89945372]
 [0.         0.54254975 0.74824537 0.87776354 0.9515185  0.97522127]
 [0.         0.55589297 0.76667154 0.89945372 0.97522127 1.        ]]


## Сравнение численных и аналитических методов

Вычислим нормы невязок для каждого из методов:

In [219]:
print('Метод простой итерации:\t{0:.6f}'.format(sla.norm(true_grid - simple_iter_grid)))
print('Метод Якоби:\t\t{0:.6f}'.format(sla.norm(true_grid - jacobi_grid)))

Метод простой итерации:	0.002103
Метод Якоби:		0.002432


## Список литературы

[1] Аристова Е.Н., Лобанов А.И. Практические занятия по вычислительной математике в МФТИ. Часть II. - М.: МФТИ, 2015. - 310 с